# FredSeries Overview

This documentation serves as a quick start guide to `FredSeries`. To review all of the `FredSeries` functionality please see the [API documentation](https://pyfredapi.readthedocs.io/en/latest/references/api.html#module-pyfredapi.api.series).

A primary use case of the FRED web service is to pull economic series observations for analysis or reporting. `FredSeries` covers all of the economic series endpoints available from the FRED API.

In [ ]:
from pyfredapi import FredSeries
from rich import print as rprint

In [ ]:
client = FredSeries()

## Get series info

Before looking at series data, it is helpful to understand the `pyfredapi`'s `get_series_info` method and `SeriesInfo` object. Each series published by FRED has associated metadata, such as the observations start and end dates, the frequency at which the series is published, and so on. You can query a series' information directly with `get_series_info`. `get_series_info` returns a `SeriesInfo` object that contains all the metadata for the given series.

In the below example, we request information on the Feds Funds Rate. From the result, we can see that the Federal Funds Rate series is available back to 1954-07-01 and is published every month.

In [ ]:
fed_funds_info = client.get_series_info(series_id="FEDFUNDS")

In [ ]:
# Using rich to pretty print the SeriesInfo
rprint(fed_funds_info)

`SeriesInfo` also provides a useful method to open the FRED webpage for the series. Call `open_url` on the info object and a new browser tab will open.

In [ ]:
fed_funds_info.open_url()

## Pull data with `FredSeries`

The `get_series` method gets the latest data available for a given series. `get_series` returns a `SeriesData` object. SeriesData is made of two components. The first is info, which is the same as the above. The second is the data itself. The data can returned as with a pandas dataframe or json.

In [ ]:
fed_funds_data = client.get_series(series_id="FEDFUNDS")

In [ ]:
fed_funds_data.info == fed_funds_info

In [ ]:
fed_funds_data.data.tail()

In [ ]:
fed_funds_data.data.dtypes

## Plot Series

If you have [plotly](https://plotly.com/python/) installed, you can use the `plot` method to build a time series plot of the data.

In [ ]:
fed_funds_data.plot()

## Data revisions

Many economic series are often revised as more complete information is made available. Every time a new version of a data series is released, FRED displays the latest version, and the replaced version is archived in [ALFRED](https://alfred.stlouisfed.org/). ALFRED stores all the previous versions of data so that it possible to understand, "what was known when?".

`pyfredapi` provides methods to access data from ALFRED:

* `get_series_all_releases` - get data for all releases
* `get_series_initial_release` - get data for the initial release
* `get_series_asof_date` - get data released on or before a specific date

### Get all releases

`get_series_all_releases` returns all the observations for each release of an economic series. In the example below, a request is made for all the releases of Gross Domestic Product. We can see that the GDP estimates for Q2 2022 has been revised 3 times. The first release was on 2022-07-28. Since then, two revisions have been released on 2022-08-25 and 2022-09-29.

In [ ]:
gdp_all_releases = client.get_series_all_releases("GDP")
gdp_all_releases.data.tail()

### Get initial release

`get_series_initial_release` return only the first release of the series. Below we see that only first estimate of Q2 2022 GDP released on 2022-07-28 is included in the dataframe.

In [ ]:
gdp_initial_release = client.get_series_initial_release("GDP")
gdp_initial_release.data.tail()

## Get releases as of a data

`get_series_asof_date` returns all releases of a series made on or before a given date. This is helpful if you want limit your analysis window to only the data available on a given date.

For example, suppose we want the GDP estimates available on or before 2022-09-01. We can use `get_series_asof_date` with the date `2022-09-01`. The response includes the Q2 2022 estimates for 2022-07-28 and 2022-08-25, but not 2022-09-29 since that is after 2022-09-01.

In [ ]:
gdp_090122 = client.get_series_asof_date("GDP", date="2022-09-01")
gdp_090122.data.tail()